# Auto reload modules

In [1]:
%load_ext autoreload
%autoreload 2

# Directories

In [2]:
import pandas as pd
import sqlite3 as db
import fs
# from fs.path import combine
# from module.dir_utils import *

# Functions

In [3]:
from fs.osfs import OSFS
from fs.subfs import SubFS

def mkdir(fs_object: OSFS, path: str) -> SubFS:
    """Ensure the directory exists; create it if it doesn't.
    
    Args:
    - fs (OSFS): An OSFS object representing the file system.
    
    Returns:
    - OSFS: The OSFS object representing the file system.
    """
    return fs_object.makedir(path) if not fs_object.exists(path) else fs_object.opendir(path)

In [4]:
def get_fulldf(SubFS: SubFS) -> pd.DataFrame:
    """
    Connects to databases and concatenate information into a DataFrame.
    
    Parameters:
    SubFS (SubFS): Object representing file system operations.
    
    Returns:
    pd.DataFrame: DataFrame containing concatenate information from databases.
    """
    df = pd.DataFrame()  # DataFrame para almacenar los datos combinados
    
    try:
        for index, path in enumerate(SubFS.walk.files()):
            # Create connection to the DB
            conn = db.connect(SubFS.getsyspath(path))
            
            # Read data from the database
            query = 'SELECT * FROM LoggedProcessValue;'
            data = pd.read_sql_query(query, conn)
            
            # Concatenate the data to df
            df = pd.concat([df, data], ignore_index=True)
            
            # Close the connection
            conn.close()
            
    except Exception as e:
        print(f"Error processing database at {path}: {str(e)}")
    
    return df

# Code

In [5]:
DB_DIR = fs.open_fs("../data/external/DataBase")
DIR_TAG_LOGGING = DB_DIR.getsyspath("HMI_RT_5_TagLoggingDatabase.db3")
DIR_TAG_HMI_RT_5_TLG1889 = DB_DIR.opendir("HMI_RT_5_TLG1889")
DIR_TAG_HMI_RT_5_TLG1893 = DB_DIR.opendir("HMI_RT_5_TLG1893")
DIR_TAG_HMI_RT_5_TLG1895 = DB_DIR.opendir("HMI_RT_5_TLG1895")
DIR_TAG_HMI_RT_5_TLG1897 = DB_DIR.opendir("HMI_RT_5_TLG1897")
DIR_TAG_HMI_RT_5_TLG1904 = DB_DIR.opendir("HMI_RT_5_TLG1904")


In [15]:
CSV_DIR = fs.open_fs("../data/processed")

# Logs files paths
CSV_DIR_LOGS = mkdir(CSV_DIR, "logs")
DIR_LOG = CSV_DIR_LOGS.getsyspath("log.csv")
DIR_LoggingTag_notna = CSV_DIR_LOGS.getsyspath("LoggingTag_notna.csv")

# Databases files paths
CSV_DIR_TLG1889 = mkdir(CSV_DIR, "TLG1889")
DIR_TLG1889 = CSV_DIR_TLG1889.getsyspath("TLG1889.csv")
CSV_DIR_TLG1893 = mkdir(CSV_DIR, "TLG1893")
CSV_DIR_TLG1895 = mkdir(CSV_DIR, "TLG1895")
CSV_DIR_TLG1897 = mkdir(CSV_DIR, "TLG1897")
CSV_DIR_TLG1904 = mkdir(CSV_DIR, "TLG1904")

In [7]:
# Create connection to the DB
conn= db.connect(DIR_TAG_LOGGING)

In [8]:
df_log = pd.read_sql_query('SELECT * from Log;',conn)
df_log.loc[:, 'table_name'] = df_log['Name'].str.split('::').str[-1]
# Save to CSV
df_log.to_csv(DIR_LOG, encoding='utf-8', index=False)

df_log


,ObjectId,Name,DisabledState,Location,TimePeriod,StorageMode,SegmentTimePeriod,BackupStrategy,SegmentStartingTime,MaxSize,SegmentMaxSize,ProductSource,Version,PrimaryBackupPath,SecondaryBackupPath,StartTime,EndTime,table_name
0,1889,HMI_RT_5::Descarga,0,D:\DataBase,103680000000000,0,6048000000000,0,132406029160000000,1000,100,None,0,,,2021-09-28 23:18:52.000,None,Descarga
1,1893,HMI_RT_5::Transferencia,0,D:\DataBase,103680000000000,0,6048000000000,0,132406029210000000,1000,100,None,0,,,2021-09-28 23:18:52.000,None,Transferencia
2,1895,HMI_RT_5::Recupero,0,D:\DataBase,103680000000000,0,6048000000000,0,132406029230000000,1000,100,None,0,,,2021-09-28 23:18:52.000,None,Recupero
3,1897,HMI_RT_5::CIP,0,D:\DataBase,103680000000000,0,6048000000000,0,132406029310000000,1000,100,None,0,,,2021-09-28 23:18:52.000,None,CIP
4,1904,HMI_RT_5::Planta,0,D:\DataBase,103680000000000,0,6048000000000,0,132406848560000000,1000,100,None,0,,,2021-09-28 23:18:52.000,None,Planta
5,2315,HMI_RT_5::Barrido,0,D:\DataBase,103680000000000,0,6048000000000,0,132417335850000000,1000,100,None,0,,,2021-09-28 23:18:52.000,None,Barrido
6,2969,HMI_RT_5::RepTrans,0,D:\DataBase,103680000000000,0,6048000000000,0,132472622670000000,1000,100,None,0,,,2021-09-28 23:18:52.000,None,RepTrans


In [9]:
df_LoggingTag = pd.read_sql_query('SELECT pk_Key, TagObjectId, LoggingTagId, fk_LogId, Name, DataType, pk_StartTime, EndTime from LoggingTag;',conn)
# df_LoggingTag.to_csv('LoggingTag.csv', encoding='utf-8', index=False)
df_LoggingTag

,pk_Key,TagObjectId,LoggingTagId,fk_LogId,Name,DataType,pk_StartTime,EndTime
0,1,1351,687865857,NaN,HMI_RT_5::FT-200-A-1-DB_oValue:FIT-Transferencia,10,132773453328968868,1.327882e+17
1,1,1351,687865857,1904.0,HMI_RT_5::FT-200-A-1-DB_oValue:FIT-Transferencia,10,132788173370000000,NaN
2,2,1360,687865857,NaN,HMI_RT_5::FT-200-A-2-DB_oValue:FIT-Descarga,10,132773453328969543,1.327882e+17
3,2,1360,687865857,1904.0,HMI_RT_5::FT-200-A-2-DB_oValue:FIT-Descarga,10,132788173370000000,NaN
4,3,1408,687865857,NaN,HMI_RT_5::LTP-300-B-1-DB_oValue:NivelSiloSQ,10,132773453328969594,1.327882e+17
...,...,...,...,...,...,...,...,...
139,70,3314,687865857,1897.0,HMI_RT_5::CIP_Pasos:cPasos,14,132788173370000000,NaN
140,71,3315,687865857,NaN,HMI_RT_5::RECType:rSec,14,132773453328974396,1.327882e+17
141,71,3315,687865857,1895.0,HMI_RT_5::RECType:rSec,14,132788173370000000,NaN
142,72,2504,687865857,NaN,HMI_RT_5::SP_Desc_PV_Temp_Desc:Temperatura,10,132774157580917529,1.327882e+17


In [10]:
pd.options.mode.copy_on_write = True 
df_LoggingTag_notna = df_LoggingTag[df_LoggingTag['fk_LogId'].notna()]
df_LoggingTag_notna.loc[:, 'tag_name'] = df_LoggingTag_notna['Name'].str.split(':').str[-1]

# Save to CSV
df_LoggingTag_notna.to_csv(DIR_LoggingTag_notna, encoding='utf-8', index=False)

df_LoggingTag_notna

,pk_Key,TagObjectId,LoggingTagId,fk_LogId,Name,DataType,pk_StartTime,EndTime,tag_name
1,1,1351,687865857,1904.0,HMI_RT_5::FT-200-A-1-DB_oValue:FIT-Transferencia,10,132788173370000000,NaN,FIT-Transferencia
3,2,1360,687865857,1904.0,HMI_RT_5::FT-200-A-2-DB_oValue:FIT-Descarga,10,132788173370000000,NaN,FIT-Descarga
5,3,1408,687865857,1904.0,HMI_RT_5::LTP-300-B-1-DB_oValue:NivelSiloSQ,10,132788173370000000,NaN,NivelSiloSQ
7,4,1417,687865857,1904.0,HMI_RT_5::LTR-400-A-1-DB_oValue:NivelTK1,10,132788173370000000,NaN,NivelTK1
9,5,1426,687865857,1904.0,HMI_RT_5::LTR-400-A-2-DB_oValue:NivelTk2,10,132788173370000000,NaN,NivelTk2
...,...,...,...,...,...,...,...,...,...
135,68,3312,687865858,1897.0,HMI_RT_5::CIPType:cSecuencia,14,132788173370000000,NaN,cSecuencia
137,69,3313,687865857,1897.0,HMI_RT_5::CIP_Operador:cOperador,14,132788173370000000,NaN,cOperador
139,70,3314,687865857,1897.0,HMI_RT_5::CIP_Pasos:cPasos,14,132788173370000000,NaN,cPasos
141,71,3315,687865857,1895.0,HMI_RT_5::RECType:rSec,14,132788173370000000,NaN,rSec


In [11]:
# Close conncection
conn.close()

## TLG1889

In [16]:
df_TLG1889 = get_fulldf(DIR_TAG_HMI_RT_5_TLG1889)
df_TLG1889.to_csv(DIR_TLG1889, encoding='utf-8', index=False)
df_TLG1889

Error processing database at /HMI_RT_5_TLG1889_20240307_171516.db3-shm: Execution failed on sql 'SELECT * FROM LoggedProcessValue;': file is not a database


,pk_TimeStamp,pk_fk_Id,Quality,Value
0,133539005217964279,12,192,6.931463e+01
1,133539005217964279,14,192,8.391804e+06
2,133539005217964279,16,192,3.681503e+04
3,133539005217964279,17,192,0.000000e+00
4,133539005217964279,18,192,0.000000e+00
...,...,...,...,...
845,133549028842115583,19,192,0.000000e+00
846,133549028842115583,20,192,0.000000e+00
847,133549028842115583,22,192,0.000000e+00
848,133549028842115583,72,192,5.219872e+01


In [13]:
df_log[df_log['ObjectId'] == 1889]['table_name']

0    Descarga
Name: table_name, dtype: object

# TLG1893